In [1]:
import numpy as np
import pandas as pd

import cv2
import mediapipe as mp

In [5]:
model_path = "D:\Documents\GitHub\CIR\emotion_classification\\face_landmarker (1).task"

In [6]:
import mediapipe as mp
import time

BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a face landmarker instance with the video mode:
options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)

cap = cv2.VideoCapture(0)

with FaceLandmarker.create_from_options(options) as landmarker:

    while True:
        ret, frame = cap.read()
        
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)
        timestamp = int(time.time() * 1000)
        results = landmarker.detect_for_video(mp_image, timestamp)
        print(results.face_blendshapes)
        
        annotated_image = draw_landmarks_on_image(image_rgb, results)
        frame = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
        
        cv2.imshow('Camera Feed', frame)
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [1]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

In [4]:
import mediapipe as mp
from mediapipe.tasks import python as mp_python

MP_TASK_FILE = "face_landmarker_with_blendshapes.task"

class FaceMeshDetector:

    def __init__(self):
        with open(MP_TASK_FILE, mode="rb") as f:
            f_buffer = f.read()
        base_options = mp_python.BaseOptions(model_asset_buffer=f_buffer)
        options = mp_python.vision.FaceLandmarkerOptions(
            base_options=base_options,
            output_face_blendshapes=True,
            output_facial_transformation_matrixes=True,
            running_mode=mp.tasks.vision.RunningMode.LIVE_STREAM,
            num_faces=1,
            result_callback=self.mp_callback)
        self.model = mp_python.vision.FaceLandmarker.create_from_options(
            options)

        self.landmarks = None
        self.blendshapes = None
        self.blendnames = None
        self.latest_time_ms = 0

    def mp_callback(self, mp_result, output_image, timestamp_ms: int):
        if len(mp_result.face_landmarks) >= 1 and len(
                mp_result.face_blendshapes) >= 1:

            self.landmarks = mp_result.face_landmarks[0]
            self.blendshapes = [b.score for b in mp_result.face_blendshapes[0]]
            self.blendnames = [b.category_name for b in mp_result.face_blendshapes[0]]

    def update(self, frame):
        t_ms = int(time.time() * 1000)
        if t_ms <= self.latest_time_ms:
            return

        frame_mp = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        self.model.detect_async(frame_mp, t_ms)
        self.latest_time_ms = t_ms

    def get_results(self):
        return self.landmarks, self.blendshapes, self.blendnames

In [1]:
import time
import cv2
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python as mp_python

MP_TASK_FILE = "face_landmarker_with_blendshapes.task"

class FaceMeshDetector:

    def __init__(self):
        with open(MP_TASK_FILE, mode="rb") as f:
            f_buffer = f.read()
        base_options = mp_python.BaseOptions(model_asset_buffer=f_buffer)
        options = mp_python.vision.FaceLandmarkerOptions(
            base_options=base_options,
            output_face_blendshapes=True,
            output_facial_transformation_matrixes=True,
            running_mode=mp.tasks.vision.RunningMode.LIVE_STREAM,
            num_faces=1,
            result_callback=self.mp_callback)
        self.model = mp_python.vision.FaceLandmarker.create_from_options(
            options)

        self.landmarks = None
        self.blendshapes = None
        self.blendnames = None
        self.latest_time_ms = 0

    def mp_callback(self, mp_result, output_image, timestamp_ms: int):
        if len(mp_result.face_landmarks) >= 1 and len(
                mp_result.face_blendshapes) >= 1:

            self.landmarks = mp_result.face_landmarks[0]
            self.blendshapes = [b.score for b in mp_result.face_blendshapes[0]]
            self.blendnames = [b.category_name for b in mp_result.face_blendshapes[0]]

    def update(self, frame):
        t_ms = int(time.time() * 1000)
        if t_ms <= self.latest_time_ms:
            return

        frame_mp = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        self.model.detect_async(frame_mp, t_ms)
        self.latest_time_ms = t_ms

    def get_results(self):
        return self.landmarks, self.blendshapes, self.blendnames

def main():
    print("Starting capture")
    facemesh_detector = FaceMeshDetector()
    cap = cv2.VideoCapture(0)

    # Wait for user to press enter
    while True:
        ret, frame = cap.read()
        cv2.putText(frame, "Press enter to start calibration", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == 13:
            break
    
    calibrating = True
    current_frame = 0
    frames = 100
    blends = {"mouthPressLeft":0, "mouthPressRight":0, "mouthSmileLeft":0, "mouthSmileRight":0}
    
    while calibrating:
        ret, frame = cap.read()  
        facemesh_detector.update(frame)
        landmarks, blendshapes, blendnames = facemesh_detector.get_results()
        if (landmarks is None) or (blendshapes is None):
            continue

        if calibrating:
            current_frame += 1
            for i, blendname in enumerate(blendnames):
                if blendname in blends.keys():
                    blends[blendname] += blendshapes[i]
            
            if current_frame > frames:
                for key in blends.keys():
                    blends[key] /= frames
                calibrating = False
        
        cv2.putText(frame, "Calibrating", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('frame', frame) 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    print(blends)
    factors = [0, 0, 0, 0]
    
    while True:
        ret, frame = cap.read()

        facemesh_detector.update(frame)
        landmarks, blendshapes, blendnames = facemesh_detector.get_results()
        if (landmarks is None) or (blendshapes is None):
            continue
        
        index = 0
        for i, blendname in enumerate(blendnames):
            if blendname in blends.keys():
                factors[index] = blendshapes[i] / blends[blendname]
                index += 1
            
        smile_degree = 0
        
        mouthPressFactor = (factors[0] + factors[1]) / 4
        # mouthPress factors:
        if mouthPressFactor > 1:
            mood = "slight smile"
        else:
            mood = "no smile"
        
        # mouthSmile factors:
        mouthSmileFactor = (factors[2] + factors[3]) / 500
        if mouthSmileFactor > 1:
            mood = "big smile"
        
        if mouthSmileFactor < 1:
            if mouthPressFactor < 1:
                smile_degree = 0
            elif mouthPressFactor < 2:
               smile_degree = 1
            elif mouthPressFactor >= 2:
               smile_degree = 2
        elif mouthSmileFactor < 2:
            smile_degree = 3
        elif mouthSmileFactor >= 2:
            smile_degree = 4
        
        print(mouthSmileFactor, mouthPressFactor)
        # mood = sum(factors)        # print(factors)
        cv2.putText(frame, f"Classifying: {mood} {smile_degree}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('frame', frame) 
        if cv2.waitKey(50) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    exit()

In [2]:
main()

Starting capture
{'mouthPressLeft': 0.08179226841777563, 'mouthPressRight': 0.03647416544146836, 'mouthSmileLeft': 0.0006661223650007742, 'mouthSmileRight': 0.000353798706564703}
0.00540492966288651 0.8635770162793158
0.004720684952824106 0.7853974423727658
0.003378556255065684 0.701952821887507
0.004780115190503165 0.8260039386027513
0.0036898599385114875 0.6386255287601734
0.003901921987142993 0.8167698651842652
0.002891997447514284 0.7487254677218219
0.00270623389905026 0.7665285455257789
0.0028755729412964144 0.7471388877236499
0.0036331212533982247 0.7736985963222355
0.003607165307783659 0.7768564028013414
0.004864901091115038 0.8358648732733404
0.003899517761252022 0.7068148086784656
0.0037864829864021187 0.7449591609112909
0.005021148621025127 0.8147708991729236
0.0036265234761619193 0.8136578613106444
0.0027262036699907694 0.7761970392202859
0.003590113274076695 0.6848290860363131
0.00362696655730413 0.7326935109532647
0.004951027431550506 0.7879712278831927
0.00305680003818484